In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.nn.init import kaiming_uniform_#
from matplotlib import pyplot as plt
import time

In [2]:
import numpy as np
import gym
from tqdm import tqdm
import random as rand
from itertools import count
from torch.distributions import Categorical
import torch.multiprocessing as mp 
main_seed = 0

In [3]:
mp.get_all_sharing_strategies()

{'file_system'}

In [4]:
pipes =[mp.Pipe() for rank in range(5) ]

In [5]:
'''
multiprocessing.Pipe([duplex])
    Returns a pair (conn1, conn2) of Connection objects representing the ends of a pipe.
    If duplex is True (the default) then the pipe is bidirectional. 
    If duplex is False then the pipe is unidirectional: 
    conn1 can only be used for receiving messages and conn2 can only be used for sending messages.
see: https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Pipe
'''

'\nmultiprocessing.Pipe([duplex])\n    Returns a pair (conn1, conn2) of Connection objects representing the ends of a pipe.\n    If duplex is True (the default) then the pipe is bidirectional. \n    If duplex is False then the pipe is unidirectional: \n    conn1 can only be used for receiving messages and conn2 can only be used for sending messages.\nsee: https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Pipe\n'

In [6]:
for i in range(len(pipes)):
    print('..............')
    print(pipes[i])

..............
(<multiprocessing.connection.PipeConnection object at 0x0000025BF02EE9C8>, <multiprocessing.connection.PipeConnection object at 0x0000025BFD688AC8>)
..............
(<multiprocessing.connection.PipeConnection object at 0x0000025BFD3AF248>, <multiprocessing.connection.PipeConnection object at 0x0000025BFD692108>)
..............
(<multiprocessing.connection.PipeConnection object at 0x0000025BFD692248>, <multiprocessing.connection.PipeConnection object at 0x0000025BFD692408>)
..............
(<multiprocessing.connection.PipeConnection object at 0x0000025BFD692348>, <multiprocessing.connection.PipeConnection object at 0x0000025BFD692488>)
..............
(<multiprocessing.connection.PipeConnection object at 0x0000025BFD6923C8>, <multiprocessing.connection.PipeConnection object at 0x0000025BFD692508>)


In [7]:
env = gym.make("CartPole-v0")
env.seed(0)
env

<TimeLimit<CartPoleEnv<CartPole-v0>>>

In [8]:
class linearApproximator_A2C(nn.Module):
    def __init__(self,state_shape,policy_outputs, state_value_output=1, hidden_dims=(32,32)):
        super(linearApproximator_A2C, self).__init__()
        self.input_size = state_shape
        self.policy_outputs = policy_outputs
        self.state_value_output = state_value_output
        self.device = torch.device("cuda" if torch.cuda.is_available()\
                                   else "cpu")
        
        self.fc1  = nn.Linear(self.input_size,hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            hidden_layer = nn.Linear(\
                                hidden_dims[i], hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        
        self.policy_output_layer  = nn.Linear(hidden_dims[-1],self.policy_outputs)
        self.state_value_output_layer = nn.Linear(hidden_dims[-1],self.state_value_output)
        self.to(self.device)
        
        
        
    def forward(self, state_shape):
        if not isinstance(state_shape, torch.Tensor):
            state_shape = torch.tensor(state_shape, dtype=torch.float32)
        state_shape = state_shape.to(self.device)
            
        x = self.fc1(state_shape)
        x = F.relu(x)
        
        for hidden_layer in self.hidden_layers:
            x = F.relu(hidden_layer(x))
        
        logits = self.policy_output_layer(x)#logits, preferences of actions
        state_value = self.state_value_output_layer(x)#predicted state value
        return logits, state_value
        
    def full_pass(self, state):
        if not isinstance(state_shape, torch.Tensor):
            state_shape = torch.tensor(state_shape, dtype=torch.float32)
        state = state.float().to(device)
        logits, state_value = self.forward(state)
        distribution = Categorical(logits=logits)
        action = distribution.sample()#sample action
        log_prob_action = distribution.log_prob(action).unsqueeze(-1)#gets prob of sampled action
        entropy = distribution.entropy().unsqueeze(-1)
        return action.item(), log_prob_action, entropy, logits, state_value

In [10]:
def select_action(action_model, state):
    action, log_prob_action, entropy, logits\
                = action_model.full_pass(state)
    return action, log_prob_action, entropy

In [11]:
'''
multiprocessing.Pipe([duplex])
    Returns a pair (conn1, conn2) of Connection objects representing the ends of a pipe.
    If duplex is True (the default) then the pipe is bidirectional. 
    If duplex is False then the pipe is unidirectional: 
    conn1 can only be used for receiving messages and conn2 can only be used for sending messages.
see: https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Pipe
'''

'\nmultiprocessing.Pipe([duplex])\n    Returns a pair (conn1, conn2) of Connection objects representing the ends of a pipe.\n    If duplex is True (the default) then the pipe is bidirectional. \n    If duplex is False then the pipe is unidirectional: \n    conn1 can only be used for receiving messages and conn2 can only be used for sending messages.\nsee: https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Pipe\n'

In [14]:
'''
multiprocessing.Pipe([duplex])
    Returns a pair (conn1, conn2) of Connection objects representing the ends of a pipe.
    If duplex is True (the default) then the pipe is bidirectional. 
    If duplex is False then the pipe is unidirectional: 
    conn1 can only be used for receiving messages and conn2 can only be used for sending messages.
see: https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Pipe
'''
class Multiprocess_env():
    def __init__(self, env, n_seeds):
        #n_seeds is basically == no of workers
        self.env = env
        self.n_seeds = n_seeds
        self.env_pipes = [mp.Pipe() for seeds in range(self.n_seeds)]
        self.envs = [mp.Process\
                    (target=self.work,\
                    args=(seed, self.env_pipes[seed][1])) for seed in range(self.n_seeds)]
        [w.start() for w in self.envs]#these envs can only send out data
        
    def broadcast_msg(self, msg):
        [receiver.send(msg) for receiver, sender in self.env_pipes]
        
    def send_msg(self, msg, seed):
        receiver, sender = self.env_pipes[seed]
        receiver.send(msg)
        
    def close(self, **kwargs):
        self.broadcast_msg(('close', kwargs))
        [e.join() for e in self.envs]
    
    def past_limit(self, **kwargs):
        self.broadcast_msg(('past_limit', kwargs))
        return np.vstack([receiver.recv()\
                             for receiver, sender in self.env_pipes])
    
    def step(self, actions):
        assert len(actions) == self.n_seeds
        [self.send_msg\
            (('step', {'action':actions[seed]}), seed)\
                        for seed in range(self.n_seeds)]
        results = []
        for seed in range(self.n_seeds):
            receiver, sender = self.env_pipes[seed]
            obj_, reward, done, info = receiver.recv()
            results.append((obj_,
                           np.array(reward, dtype=np.float),
                           np.array(done, dtype=np.float),
                           info))
        return[np.vstack(block) for block in np.array(results).T]
        
    def reset(self, seed=None, **kwargs):
        if seed is not None:
            receiver, sender = self.env_pipes[seed]
            self.send_msg(('reset', {}), seed)
            o = receiver.recv()
            return o
        
        self.broadcast_msg(('reset',kwargs))
        return np.vstack([receiver.recv()\
                             for receiver, sender in self.env_pipes])
            
    def work(self, seed, sender_pipe):
        env = self.env
        env.seed(seed)
        while True:
            cmd, kwargs = sender_pipe.recv()
            if cmd == 'reset':
                sender_pipe.send(env.reset(**kwargs))
            elif cmd == 'step':
                sender_pipe.send(env.step(**kwargs))
            elif cmd == 'past_limit':
                sender_pipe.send(env._elapsed_steps >= env._max_episode_steps)
            else:
                env.close(**kwargs)
                del env
                sender_pipe.close()
                break

In [15]:
class A2C():
    def __init__(self, a2c_network,
                a2c_optimizer,
                a2c_lr,
                policy_loss_weight,
                value_loss_weight,
                entropy_loss_weight,
                max_steps,
                n_seeds,
                tau,
                gamma):
        assert n_seeds > 1
        self.a2c_network = a2c_network
        self.a2c_optimizer = a2c_optimizer
        self.a2c_lr = a2c_lr
        
        self.policy_loss_weight = policy_loss_weight
        self.entropy_loss_weight = entropy_loss_weight
        self.value_loss_weight = value_loss_weight
        
        self.max_steps = max_steps
        self.n_seeds = n_seeds
        self.tau = tau
        self.gamma = gamma
        
    def optimize_model(self, log_pa, entropies, states, rewards):
        log_pa = torch.stack(log_pa).squeeze()
        entropies = torch.stack(entropies).squeeze()
        states = torch.stack(states).squeeze()
        
        T = len(rewards)#trajectory length
        discounts = np.logspace(0, T, num = T,
                               base = self.gamma, endpoint=False)
        rewards = np.array(rewards).squeeze()
        

In [ ]:
 abort_after = 60
    start = time.time()
    terminate_env = False
    ovr_rewards = []
    
    while not terminate_env:
        delta = time.time() - start
        state = env.reset()#set of n_envs
        acc_rewards = 0
        
        n_steps = 0
        reward_state = []
        state_store = []
        log_pa = []
        entropy_store = []
        
        for step in count(start = 1):
            action, log_prob, entropy = \
                    select_action(A2C_policy_network, state)
            next_state, reward, done, info = \
                    env.step(action)
            acc_rewards += reward
            is_truncated = 'TimeLimit.truncated' in info and\
                                info['TimeLimit.truncated']
            is_failure = done and not is_truncated
            reward_store.append(reward)
            state_store.append(state)
            log_pa.append(log_prob)
            entropy_store.append(entropy)
            state = next_state
            
            if done==True or step - n_steps == max_steps:
                if is_failure:
                    reward_store[-1] = 0.0
                if len(reward_store) < 2:
                    continue
                optimize_model(reward_state, log_pa,\
                    state_store, entropy_store, A2C_value_network,\
                    A2C_policy_network, A2C_policy_optimizer,\
                    A2C_value_optimizer, gamma, entropy_beta,\
                    gae_tau)
                reward_store = []
                state_store = []
                log_pa = []
                entropy_store = []
                n_steps = step
            if done == True:
                ovr_rewards.append(acc_rewards)
                break
            if delta >= abort_after:
                terminate_worker = True
                break
    return ovr_rewards

In [ ]:
def A2C_(env,
        n_envs,
        gamma=0.99):
    
    obs_space = len(env.reset())
    action_space = len(env.action_space.high)
    A2C_network = linearApproximator_A2C(obs_space, action_space)
    A2C_network_optimizer = torch.optim.Adam(A2C_network.parameters(),lr=0.0008, weight_decay = 0.01)
    
    for seed in range(n_envs):
        #get trajectory from environment seed using model
        
        #optimize model
        